In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv(r'C:\Users\Professional\Desktop\Kaggle competitions\30-days-of-ml\data\train.csv')
test = pd.read_csv(r'C:\Users\Professional\Desktop\Kaggle competitions\30-days-of-ml\data\test.csv')
sample=pd.read_csv(r'C:\Users\Professional\Desktop\Kaggle competitions\30-days-of-ml\data\sample_submission.csv')

In [4]:
object_col=[col for col in train.columns if 'cat' in col]

In [6]:
xgb_params = {
    #'tree_method':'gpu_hist',         ## parameters for gpu
    #'gpu_id':0,                       #
    #'predictor':'gpu_predictor',      #
    'n_estimators': 10000,
    'learning_rate': 0.03628302216953097,
    'subsample': 0.7875490025178415,
    'colsample_bytree': 0.11807135201147481,
    'max_depth': 3,
    'booster': 'gbtree', 
    'reg_lambda': 0.0008746338866473539,
    'reg_alpha': 23.13181079976304,
    'n_jobs':-1,
    'random_state':40
}

In [8]:
model=XGBRegressor(**xgb_params)
i=0
sub=[]
kf=KFold(n_splits=10,shuffle=True,random_state=42)
for train_index,test_index in kf.split(train):
    i+=1
    xtrain=train.iloc[train_index]
    xvalid=train.iloc[test_index]
    
    df_test=test.copy()
    
    encoder=OrdinalEncoder()
    
    xtrain[object_col]=encoder.fit_transform(xtrain[object_col])
    xvalid[object_col]=encoder.transform(xvalid[object_col])
    df_test[object_col]=encoder.transform(df_test[object_col])
    
    model.fit(xtrain.drop('target',axis=1),xtrain['target'],
            early_stopping_rounds=300,
            eval_set=[(xvalid.drop('target',axis=1), xvalid['target'])], 
            verbose=2000)
    
    pred_valid=model.predict(xvalid.drop('target',axis=1))
    pred_train=model.predict(xtrain.drop('target',axis=1))
    print(f'fold {i} validation error ',mean_squared_error(xvalid['target'],pred_valid,squared=False))
    print(f'fold {i} training error ',mean_squared_error(xtrain['target'],pred_train,squared=False))
    print("--------------------")
    
    pred=model.predict(df_test)
    sub.append(pred)

[0]	validation_0-rmse:7.50941
[2000]	validation_0-rmse:0.71929
[4000]	validation_0-rmse:0.71666
[6000]	validation_0-rmse:0.71600
[7666]	validation_0-rmse:0.71588
fold 1 validation error  0.7158716239908846
fold 1 training error  0.7064129061666355
--------------------
[0]	validation_0-rmse:7.49111
[2000]	validation_0-rmse:0.71909
[4000]	validation_0-rmse:0.71634
[6000]	validation_0-rmse:0.71563
[7870]	validation_0-rmse:0.71546
fold 2 validation error  0.7154524993216206
fold 2 training error  0.7061455927211416
--------------------
[0]	validation_0-rmse:7.49705
[2000]	validation_0-rmse:0.71736
[4000]	validation_0-rmse:0.71521
[6000]	validation_0-rmse:0.71475
[6481]	validation_0-rmse:0.71474
fold 3 validation error  0.7147371228358378
fold 3 training error  0.7077744507041918
--------------------
[0]	validation_0-rmse:7.49705
[2000]	validation_0-rmse:0.72050
[4000]	validation_0-rmse:0.71786
[6000]	validation_0-rmse:0.71710
[7985]	validation_0-rmse:0.71685
fold 4 validation error  0.7168

In [ ]:
predictions=np.mean(np.column_stack(sub),axis=1)
sample['target']=predictions

sample.to_csv('sub.csv',index=False)

